In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.feature_selection import chi2
import pyepsg
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tools
import h5py

In [48]:
#importing data

file = '/Users/tommy/documents/Database Design/Final_Proj/gun-violence-data_01-2013_03-2018.csv'

shooting_data = pd.read_csv(file)

shooting_data = shooting_data

text_data = shooting_data['incident_characteristics'].map(str) + ' ' + shooting_data['notes'].map(str)

death_data = shooting_data['n_killed']

print(max(death_data))

shooting_data.head()

50


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


# Cleaning up data

Removing stopwords such as and, or, a, etc.

Removing any non-letter elements (-,/,:, etc.)

Creating a corpus to run TF-IDF on

In [49]:
def clean_data(text_data):
    
    corpus = []
    for i in range(len(text_data)):
        text_data[i] = re.sub('[^a-zA-Z]', ' ', text_data[i])
        text_data[i] = text_data[i].lower()
        text_data[i] = text_data[i].split()
        ps = PorterStemmer()
        text_data[i] = [ps.stem(word) for word in text_data[i] if not word in set(stopwords.words('english'))]
        text_data[i] = ' '.join(text_data[i])
        corpus.append(text_data[i])
    return text_data, corpus

text_data, corpus = clean_data(text_data)

# Preprocessing the Data to TF-IDF

Tf-idf stands for term frequency-inverse document frequency, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. (http://www.tfidf.com/)

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)

IDF(t) = log_e(Total number of documents / Number of documents with term t in it)

In [50]:
tfidf = TfidfVectorizer()
tfidf.fit(text_data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [81]:
X = text_data
y = death_data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)


shot wound injur shot dead murder accident suicid mass shoot victim injur kill exclud subject suspect perpetr one locat gang involv four shot one kill unidentifi shooter getaway car


In [52]:
def accuracy_summary(pipeline, X_train, y_train, X_test, y_test):
    sentiment_fit = pipeline.fit(X_train, y_train)
    y_pred = sentiment_fit.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("accuracy score: {0:.2f}%".format(accuracy*100))
    return accuracy

# This is using SKLearn's random tree classifier to assist in determining Max Feature 

This block of code is determining the most accurate Max Feature variable to pass into the LSTM by using Random Forest, a simpler machine learning 

In [53]:
cv = CountVectorizer()
rf = RandomForestClassifier(class_weight="balanced")
n_features = np.arange(10000,25001,5000)
def nfeature_accuracy_checker(vectorizer=cv, n_features=n_features, stop_words=None, ngram_range=(1, 1), classifier=rf):
    result = []
    print(classifier)
    print("\n")
    for n in n_features:
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print("Test result for {} features".format(n))
        nfeature_accuracy = accuracy_summary(checker_pipeline, X_train, y_train, X_test, y_test)
        result.append((n,nfeature_accuracy))
    return result

tfidf = TfidfVectorizer()

print("Result for trigram with stop words (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(vectorizer=tfidf,ngram_range=(1, 3))

Result for trigram with stop words (Tfidf)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators='warn', n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)


Test result for 10000 features


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy score: 98.02%
Test result for 15000 features
accuracy score: 98.00%
Test result for 20000 features
accuracy score: 97.91%
Test result for 25000 features
accuracy score: 97.84%


# Creating test sequences

In [55]:
#after finding the most accurate feature amount from the random forest classifier
max_features = 15000
#tokenizing text to numerical values (tokens)
tokenizer = Tokenizer(nb_words=max_features, split=' ')
tokenizer.fit_on_texts(text_data.values)
#creating randomized text sequences from the text data
X1 = tokenizer.texts_to_sequences(text_data.values)
#padding the sequences so they are the same length
X1 = pad_sequences(X1)
Y1 = pd.get_dummies(death_data).values
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1, random_state = 42)
print(X1_train.shape,Y1_train.shape)
print(X1_test.shape,Y1_test.shape)

/usr/local/lib/python3.7/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


(179757, 107) (179757, 16)
(59920, 107) (59920, 16)


# Constructing the graph of the model

Creating a 3 layer RNN with a LSTM hidden layer in the middle.

This is a sequence to vector model, input tokenized sequence and returns a vector of probabilities and results.

The first layer embeds the sequences into vectors that can be passed into the LSTM layer.

The LSTM layer contains weights and biases like any Recursive Neural Network, however, is has a unique alpha between each node that determines "the memory" of each node. The sequences are viewed in a time-series fashion, where each proceeding word is weighted and then added to the result of the previously weighted words. Also an LSTM utilizes skip-connections in order to address the vanishing/exploding gradient problem.

The final layer is the output layer.

In [58]:
embed_dim = 150
lstm_out = 200
model = Sequential()
#X1 is padded so the input length is consistent for the entire structure
model.add(Embedding(max_features, embed_dim,input_length = X1.shape[1]))
model.add(LSTM(lstm_out, dropout_U=0.2,dropout_W=0.2))
#final output layer of the RNN
#16 is derived the max value from the number of deaths
#in this case # of deaths is considered a class
model.add(Dense(16,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 107, 150)          4500000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 200)               280800    
_________________________________________________________________
dense_5 (Dense)              (None, 16)                3216      
Total params: 4,784,016
Trainable params: 4,784,016
Non-trainable params: 0
_________________________________________________________________
None


# Training the model

In [59]:
#the batch size is on the lower end because the model converges pretty quickly
#only needed 3 epochs because the accuracy began high and converged quickly

batch_size = 32
model.fit(X1_train, Y1_train, nb_epoch = 3, batch_size=batch_size, verbose = 2)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/3
 - 922s - loss: 0.0941 - acc: 0.9773
Epoch 2/3
 - 948s - loss: 0.0685 - acc: 0.9832
Epoch 3/3
 - 952s - loss: 0.0624 - acc: 0.9839


In [60]:
score,acc = model.evaluate(X1_test, Y1_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.07
acc: 0.98


# Checking the accuracy of the model

Using the model.predict function to pass in the test text (X1_test) and comparing the result to that of the test result in the database

In [69]:
pos_cnt, pos_correct = 0, 0

result_array, test_array = np.zeros(len(X1_test)), np.zeros(len(X1_test))

for x in range(len(X1_test)):
    
    result = model.predict(X1_test[x].reshape(1,X1_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    #saving prediction results vs. test results to create visualization
    result_array[x] = np.argmax(result)
    test_array[x] = np.argmax(Y1_test[x])
    
    if np.argmax(result) == np.argmax(Y1_test[x]):
    
            pos_correct += 1


print("pos_acc", pos_correct/len(X1_test)*100, "%")


pos_acc 98.38618157543391 %


In [70]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


# Visualizing the results of the model

In [84]:
a = np.empty(len(result_array))
b = np.arange(0, len(result_array), 1)
ind = np.arange(len(a))
np.put(a, ind, b)

tools.set_credentials_file(username='thomas_champ', api_key='fn1EyslX0oJ5MGRcWSpe')

# Create a trace
Model_Results = go.Scatter(
    x = a,
    y = result_array
)

Test_Results = go.Scatter(
    x = a,
    y = test_array
)

data = [Model_Results, Test_Results]

py.iplot(data, filename='basic-scatter')

The draw time for this plot will be slow for all clients.


In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")